# The age of spatial signatures

In [1]:
import geopandas
import dask_geopandas
from pyogrio import read_dataframe

uprn_p = '/home/jovyan/data/uk_os_openuprn/osopenuprn_202210.gpkg'
epc_p = '/home/jovyan/data/uk_epc_certificates/'

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


In [2]:
import dask.dataframe as ddf
from dask.distributed import LocalCluster, Client

client = Client(LocalCluster(n_workers=16))
client

2022-12-15 16:05:34,361 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/work/Desktop/dask-worker-space/worker-ihcldg3w', purging
2022-12-15 16:05:34,366 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/work/Desktop/dask-worker-space/worker-jlrlm4g0', purging
2022-12-15 16:05:34,368 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/work/Desktop/dask-worker-space/worker-ypo31ont', purging
2022-12-15 16:05:34,372 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/work/Desktop/dask-worker-space/worker-79wdqq8w', purging
2022-12-15 16:05:34,374 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/work/Desktop/dask-worker-space/worker-xaxbdjkc', purging
2022-12-15 16:05:34,379 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/work/Desktop/dask-worker-space/worker-b924crmc', purging
2022-12-15 16:05

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 16,Total memory: 62.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38125,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 62.49 GiB
Comm: tcp://127.0.0.1:33817,Total threads: 1
Dashboard: http://127.0.0.1:46015/status,Memory: 3.91 GiB
Nanny: tcp://127.0.0.1:39927,


## Data

### EPC certificates

In [9]:
dtypes = {
    'ADDRESS3': 'object',
   'CO2_EMISS_CURR_PER_FLOOR_AREA': 'float64',
   'ENERGY_CONSUMPTION_POTENTIAL': 'float64',
   'FLAT_TOP_STOREY': 'object',
   'HEATING_COST_CURRENT': 'float64',
   'HEATING_COST_POTENTIAL': 'float64',
   'HOT_WATER_COST_CURRENT': 'float64',
   'HOT_WATER_COST_POTENTIAL': 'float64',
   'LIGHTING_COST_CURRENT': 'float64',
   'LIGHTING_COST_POTENTIAL': 'float64',
   'MAIN_HEATING_CONTROLS': 'object',
   'NUMBER_OPEN_FIREPLACES': 'float64',
   'UPRN': 'str'
}
certs_all = ddf.read_csv(f'{epc_p}*/certificates.csv', dtype=dtypes)

In [10]:
%%time
tokeep = [
    'LMK_KEY',
    'CONSTRUCTION_AGE_BAND',
    'UPRN'
]
certs = certs_all[tokeep].compute()

/opt/conda/lib/python3.9/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.9/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.9/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.9/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/conda/lib/python3.9/site-packages/dask/dataframe/io/csv.py:181: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/opt/

CPU times: user 41.1 s, sys: 5.74 s, total: 46.8 s
Wall time: 1min 17s


### UPRN coords

In [5]:
%%time
uprn = read_dataframe(uprn_p)
uprn['UPRN'] = uprn['UPRN'].astype(str)

CPU times: user 1min 50s, sys: 9.85 s, total: 2min
Wall time: 2min 3s


In [7]:
%%time
uprn_d = dask_geopandas.read_file(uprn_p, npartitions=16)
uprn_d['UPRN'] = uprn_d['UPRN'].astype(str)
uprn = uprn_d.compute()

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /o

CPU times: user 45.3 s, sys: 12.1 s, total: 57.3 s
Wall time: 1min 39s


### Merge

---

**DIGRESSION**: comparison with `polars`

In [11]:
%%time
# Pandas
db = uprn[
    ['UPRN', 'geometry']
].join(
    certs.set_index('UPRN'), 
    on='UPRN', 
    how='inner'
)

CPU times: user 42.1 s, sys: 3.63 s, total: 45.7 s
Wall time: 44.1 s


In [12]:
uprn[['UPRN', 'geometry']].to_parquet('/home/jovyan/tmp1.pq')
certs.set_index('UPRN').to_parquet('/home/jovyan/tmp2.pq')

/tmp/ipykernel_3018380/3438183864.py:3: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  uprn[['UPRN', 'geometry']].to_parquet('/home/jovyan/tmp1.pq')


In [1]:
import polars

In [2]:
%%time
# Polars
tst = (
    polars.read_parquet('/home/jovyan/tmp1.pq')
    .lazy()
    .join(
        polars.read_parquet('/home/jovyan/tmp2.pq').lazy(), 
        on='UPRN', 
        how='inner'
    )
).collect()

CPU times: user 34.2 s, sys: 7.64 s, total: 41.8 s
Wall time: 9.8 s


---

In [3]:
%%time
pq_p = '/home/jovyan/epc_uprn.pq'
try:
    db = geopandas.read_parquet(pq_p)
except:
    db = uprn[
        ['UPRN', 'geometry']
    ].join(
        certs.set_index('UPRN'), 
        on='UPRN', 
        how='inner'
    )
    db.to_parquet(pq_q)

CPU times: user 18.7 s, sys: 7.89 s, total: 26.6 s
Wall time: 22.3 s


## Spatial Signatures

In [4]:
%%time
ss = read_dataframe(
    '/home/jovyan/data/spatial_signatures/spatial_signatures_GB.gpkg'
)

CPU times: user 1.54 s, sys: 1.03 s, total: 2.57 s
Wall time: 3.73 s


This is very detailed, which makes things much slower to run, so we simplify first:

In [5]:
%%time
sss = ss.simplify(10)

CPU times: user 1min 15s, sys: 1.11 s, total: 1min 16s
Wall time: 1min 11s


## Spatial Join

`spatialpandas` seems to have the most performant point-in-polygon, mostly due to parallelism through Dask. To join our UPRN sites to the Spatial Signature polygon where they fall into, we need to first convert both geometries (SS polygons and UPRN points) to its own format.

In [6]:
from spatialpandas import sjoin as ssjoin, GeoDataFrame as SDF

The easy (small) one is the polygon layer:

In [7]:
%time sp_ss = SDF(ss.assign(geometry=sss))

/opt/conda/lib/python3.9/site-packages/spatialpandas/geometry/polygon.py:41: ShapelyDeprecationWarning: Accessing the 'ctypes' attribute is deprecated, and will not be possible any more in Shapely 2.0
  exterior = np.asarray(shape.exterior.ctypes)
/opt/conda/lib/python3.9/site-packages/spatialpandas/geometry/polygon.py:46: ShapelyDeprecationWarning: Accessing the 'ctypes' attribute is deprecated, and will not be possible any more in Shapely 2.0
  interior = np.asarray(ring.ctypes)


CPU times: user 9.85 s, sys: 642 ms, total: 10.5 s
Wall time: 9.79 s


Then we ship the certificates to `spatialpandas` geometries. We use Dask to do the conversion of several points in parallel at the same time:

In [8]:
%%time
sp_db = (
    ddf.from_pandas(db, npartitions=16)
    .map_partitions(SDF)
).compute()

/opt/conda/lib/python3.9/site-packages/spatialpandas/geometry/line.py:33: ShapelyDeprecationWarning: Accessing the 'ctypes' attribute is deprecated, and will not be possible any more in Shapely 2.0
  return np.asarray(shape.ctypes)
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/co

CPU times: user 1min 1s, sys: 12.5 s, total: 1min 13s
Wall time: 2min 49s


And distribute across Dask:

In [ ]:
client.restart()

In [10]:
%time sp_ddb = ddf.from_pandas(sp_db, npartitions=16)

CPU times: user 10min 42s, sys: 3min 43s, total: 14min 26s
Wall time: 11min 10s


Then we're ready to do the spatial join (and write to disk to save the computation):

In [11]:
client.restart()

2022-12-15 16:22:55,702 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:22:55,705 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:22:58,892 - distributed.worker_memory - WARNING - Worker exceeded 95% memory budget. Restarting
2022-12-15 16:23:00,040 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:23:00,042 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:23:00,042 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:23:00,043 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:23:00,044 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:23:00,044 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:23:00,045 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:23:00,047 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:23:00,048 - distributed.nanny - WARNING - Restarting worker
2022-12-15 16:23:00,049 - distributed.nanny - WARNING - Restarting worker
20

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 16,Total memory: 62.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38125,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: 18 minutes ago,Total memory: 62.49 GiB
Comm: tcp://127.0.0.1:33103,Total threads: 1
Dashboard: http://127.0.0.1:41241/status,Memory: 3.91 GiB
Nanny: tcp://127.0.0.1:39927,


In [ ]:
%time tab = ssjoin(sp_ddb, sp_ss).compute()

tab2.drop(
    ['index_right', 'geometry'], axis=1
).to_parquet(
    '/home/jovyan/epc_uprn_ss.pq'
)

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /o

In comparisons, this appeared as the best approach over single core `spatialpandas` and `geopandas`.

## Analysis